# GoFundMe data preparation and EDA

The following set of commands is an outline of the data preparation for the GoFundMe dataset. The imported `csv` file is the raw data that was mined from the GoFundMe website, where the only processing conducted was the tokenization of the campaign description under the `story_tokens` column.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('gofund_data.csv')

## Processing raised and target fields

In order to perform any kind of calculation, the raw string that was scraped needs to undergo some preparation. In this case, the currency symbols that came with the dollar amounts for `raised` and `target` columns need to be
removed, before they can be converted into the appropriate data type.

`df[['raised', 'target']].head()`
```
raised	target
0	$695	$1,000
1	$695	$300
2	$690	$1,000
3	$690	$500
4	$690	$1,000
```

In [ ]:
## Remove currency symbols and replace with ''
df['raised'] = df['raised'].str.replace('€', '')
df['target'] = df['target'].str.replace('€', '')
df['raised'] = df['raised'].str.replace('$', '')
df['target'] = df['target'].str.replace('$', '')
df['raised'] = df['raised'].str.replace(',', '')
df['target'] = df['target'].str.replace(',', '')

```
	raised	target
0	695	1000
1	695	300
2	690	1000
3	690	500
4	690	1000
```

The next step is to convert the `dtype` from `object` into `int`. Here is output before being converted. Notice the difference in `dtype`.


```
Unnamed: 0        int64
url_index        object
title            object
raised           object
target           object
m_campaign       object
created_date     object
donors          float64
shares          float64
followers       float64
scrape_date      object
c_story          object
story_tokens     object
dtype: object
```

In [ ]:
# Convert dtypes
df['raised'] = df['raised'].astype(int)
df['target'] = df['target'].astype(int)
df['scrape_date'] = pd.to_datetime(df['scrape_date'])

Now, both `raised` and `target` have `int64` as `df.dtypes`
```
Unnamed: 0               int64
url_index               object
title                   object
raised                   int64
target                   int64
m_campaign              object
created_date            object
donors                 float64
shares                 float64
followers              float64
scrape_date     datetime64[ns]
c_story                 object
story_tokens            object
dtype: object
```

## Clean created_date field

The `created_date` field needs some processing prior to being converted into `datetime` objects as well. Notice how it is encoded as `dtype(object)`, compared to the `scrape_date` field. This is because new campaigns relative to the `scrape_date` were labelled as being `"Created {x} days ago"` on the GoFundMe website, whereas the `scrape_date` was directly coded as a `datetime` object during the scrape.

```
6030            2020-08-14
5631            2020-08-14
2964    Created 2 days ago
4106    Created 2 days ago
3011    Created 2 days ago
4337    Created 3 days ago
6907    Created 3 days ago
4058    Created 3 days ago
4155    Created 3 days ago
4076    Created 4 days ago
6862    Created 4 days ago
6864    Created 4 days ago
6989    Created 4 days ago
3967    Created 6 days ago
3946    Created 7 days ago
3945    Created 7 days ago
4100    Created 7 days ago
4185    Created 7 days ago
4216    Created 7 days ago
4335    Created 7 days ago
Name: created_date, dtype: object
```

In [ ]:
# Regex pattern to find the beginning of a string that is a non-digit
pattern = r'(^[\D])'

# Placeholder for lists to be used later.
days_ago = []
days_index = []
days_int = []

df_tmp = df['created_date'].str.match(pattern)

A temporary `Series` is created to identify rows where the regex match returns `True`.

```
0       False
1       False
2       False
3       False
4       False
        ...  
7072    False
7073    False
7074    False
7075    False
7076    False
Name: created_date, Length: 7077, dtype: bool
```

In [ ]:
def update_created_date():
    """Cleaning of 'Created x days ago' """
    
    for index, value in df_tmp.items():
        if value == True:
            days_index.append(index) # Append indices for rows that returned True

    for i in days_index:
        # Split "Created..." string in order to grab just the number of days        
        split_date = df['created_date'].loc[i].split(" ")
        days_int.append(int(split_date[1])) # Append just the number of days to list

update_created_date()

Now, the numbers from `"Created {x} days ago"` have been extracted and added to a list which can be used to calculate the actual date the campaign was created.

```
Extracted number list: 
[2, 2, 7, 7, 6, 3, 4, 7, 2, 3, 7, 7, 7, 3, 4, 4, 3, 4] 

List of indices associated with the row: 
[2964, 3011, 3945, 3946, 3967, 4058, 4076, 4100, 4106, 4155, 4185, 4216, 4335, 4337, 6862, 6864, 6907, 6989]
```

### SettingWithCopyWarning
The following line of code sets the `SettingWithCopyWarning` to `None` to silence the warning message pertaining to the issue of *chained assignments*.

In [ ]:
# Fix (silence) SettingWithCopyWarning!
pd.set_option('mode.chained_assignment', None)

The next block of code is a work around for the above mentioned issue with the `SettingWithCopyWarning`. Using the proper syntax according to [this](https://www.dataquest.io/blog/settingwithcopywarning/) causes some other unintended issue with whole lists being assigned to rows. So while it is advised not to silence such *false positives*, in order to move forward with this project, a decision had to be made for a workable solution, while in search for a *purist* solution.

In [ ]:
x = 0
for i in days_index:
    df['created_date'].loc[i] = str(days_int[x])
    x += 1

Now, the actual number has been extracted from the *"Created...ago"* string.

```
447              2020-08-13
2221             2020-08-13
1143             2020-08-13
3847             2020-08-13
441              2020-08-13
2563             2020-08-13
2706             2020-08-13
7060    2020-08-13 00:00:00
6963    2020-08-13 00:00:00
4251    2020-08-13 00:00:00
3935    2020-08-13 00:00:00
7034    2020-08-13 00:00:00
6030             2020-08-14
5631             2020-08-14
6031             2020-08-14
4337                      3
4155                      3
6907                      3
4058                      3
6862                      4
4076                      4
6864                      4
6989                      4
3967                      6
4185                      7
4216                      7
4100                      7
4335                      7
3946                      7
3945                      7
Name: created_date, dtype: object
```

### Convert date fields into proper datetime objects

In [ ]:
from datetime import timedelta

With the `created_date` field properly formatted, we can now calculate the actual date the campaign was launched on the website, and assign that to the specific element.

In [ ]:
# subtract days_ago from scrape_date
date_list = []
for i in days_index:
    df['created_date'].loc[i] = df['scrape_date'][i] - timedelta(days=int(df['created_date'][i]))

In [ ]:
df['created_date'] = pd.to_datetime(df['created_date'])

We now have a properly formatted date column where EDA can be applied!

```
7060   2020-08-13
3675   2020-08-13
441    2020-08-13
3935   2020-08-13
2221   2020-08-13
7034   2020-08-13
6963   2020-08-13
447    2020-08-13
3847   2020-08-13
6031   2020-08-14
6030   2020-08-14
5631   2020-08-14
4216   2020-08-14
3946   2020-08-14
4185   2020-08-14
3945   2020-08-14
4100   2020-08-14
4335   2020-08-14
3967   2020-08-15
6989   2020-08-17
6862   2020-08-17
6864   2020-08-17
4076   2020-08-17
4337   2020-08-18
4155   2020-08-18
4058   2020-08-18
6907   2020-08-18
2964   2020-08-19
4106   2020-08-19
3011   2020-08-19
Name: created_date, dtype: datetime64[ns]

Unnamed: 0               int64
url_index               object
title                   object
raised                   int64
target                   int64
m_campaign              object
created_date    datetime64[ns]
donors                 float64
shares                 float64
followers              float64
scrape_date     datetime64[ns]
c_story                 object
story_tokens            object
dtype: object
```

In [ ]:
df.head(10)

## Descriptive Statistics for the GoFundMe dataset

Now that all the variable have been properly formatted, we can perform some exploratory data analysis on the dataset!

Interestingly enough, we can actually say that almost all followers were actually donating to their chosen campaign!

In [ ]:
df.plot.scatter(x='donors', y='followers')

### Amount Raised by Category

Combining all the total amount raised for each category, we then sort them according to the category with the highest dollar amount raised so far...

In [ ]:
df_rc = pd.melt(df, id_vars=['m_campaign'], value_vars='raised', value_name='Amount Raised', var_name='Category')
df_rc = df_rc.groupby('m_campaign').sum()
df_rc.rename_axis('Category', inplace=True)
pd.options.display.float_format = '{:,.2f}'.format
df_rc = df_rc.sort_values('Amount Raised', ascending=False)
df_rc

And we can see how skewed the data is by the top 5 campaign categories.

In [ ]:
df_rc.plot(kind='bar')

### Number of Campaigns per Category

In [ ]:
df_cat = df[['m_campaign', 'title']]
df_cat = df_cat.groupby('m_campaign').count()
df_cat.drop(index='discover', inplace=True)
df_cat.rename_axis('Category', inplace=True)
df_cat = df_cat.rename(columns={'title': 'Number of Campaigns'})
df_cat = df_cat.sort_values(by='Number of Campaigns', ascending=False)
df_cat

Remember our top 5 campaign category in terms of dollar amount raised...
```
competition-fundraiser	1001113846
newlywed-fundraiser	266540890
travel-fundraiser	97447367
medical-fundraiser	18654746
emergency-fundraiser	11709363
```
As it turned out, we have a slightly different picture when the data it pitted against the number of campaigns *within* each category...

```
medical-fundraiser	584
community-fundraiser	574
volunteer-fundraiser	458
emergency-fundraiser	450
family-fundraiser	437
```

Objectively, people who donate to competitive causes are able to raise higher dollar amounts judging by the ratio of total amount raised versus total number of campaigns in each category.

In [ ]:
df_cat.plot.bar()

### Visualize Amount Raised vs Number of Donors

In [ ]:
df_rv = pd.DataFrame(df[['m_campaign', 'donors', 'raised']])
df_rv